In [64]:
import cv2
import numpy as np

# read image
imgopen = cv2.imread('../doho.png')
imgclosed = cv2.imread('../dchc.png')
tmplt_handle = cv2.imread('template_open2.png', cv2.IMREAD_UNCHANGED)
tmplt_deadbolt = cv2.imread('template_deadbolt.png', cv2.IMREAD_UNCHANGED)


def getMatch(img, tmplt):

# read template with alpha
    hh, ww = tmplt.shape[:2]
    tmplt_mask = tmplt[:,:,3]
    tmplt_mask = cv2.merge([tmplt_mask,tmplt_mask,tmplt_mask])
    tmplt2 = tmplt[:,:,0:3]
    corrimg = cv2.matchTemplate(img,tmplt2,cv2.TM_CCORR_NORMED, mask=tmplt_mask)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(corrimg)
    max_val_ncc = '{:.3f}'.format(max_val)
    print("correlation match score: " + max_val_ncc)
    xx = max_loc[0]
    yy = max_loc[1]
    print('xmatch =',xx,'ymatch =',yy)

    # draw red bounding box to define match location
    result = img.copy()
    pt1 = (xx,yy)
    pt2 = (xx+ww, yy+hh)
    cv2.rectangle(result, pt1, pt2, (0,0,255), 1)
    newimg = img[pt1[1]:pt2[1], pt1[0]:pt2[0]]
    newimg = newimg[len(newimg)//3:2*len(newimg)//3, len(newimg[0])//3:2*len(newimg[0])//3]
    # cv2.imshow('result', result)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    return newimg

# save results
ohandle = getMatch(imgopen, tmplt_deadbolt)
chandle = getMatch(imgclosed, tmplt_deadbolt)
cv2.imwrite('cd.png',chandle)


correlation match score: 0.998
xmatch = 446 ymatch = 194
correlation match score: 0.986
xmatch = 434 ymatch = 160


True

In [58]:
from skimage.metrics import structural_similarity
import cv2
import numpy as np

first = ohandle
second = chandle

# Convert images to grayscale
first_gray = cv2.cvtColor(first, cv2.COLOR_BGR2GRAY)
second_gray = cv2.cvtColor(second, cv2.COLOR_BGR2GRAY)

# Compute SSIM between two images
score, diff = structural_similarity(first_gray, second_gray, full=True)
print("Similarity Score: {:.3f}%".format(score * 100))

# The diff image contains the actual image differences between the two images
# and is represented as a floating point data type so we must convert the array 
# to 8-bit unsigned integers in the range [0,255] before we can use it with OpenCV
diff = (diff * 255).astype("uint8")

# Threshold the difference image, followed by finding contours to
# obtain the regions that differ between the two images
thresh = cv2.threshold(diff, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
contours = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = contours[0] if len(contours) == 2 else contours[1]

# Highlight differences
mask = np.zeros(first.shape, dtype='uint8')
filled = second.copy()


Similarity Score: 79.355%


In [ ]:
from sentence_transformers import SentenceTransformer, util
from PIL import Image
import glob
import os

# Load the OpenAI CLIP Model
print('Loading CLIP Model...')
model = SentenceTransformer('clip-ViT-B-32')

# Next we compute the embeddings
# To encode an image, you can use the following code:
# from PIL import Image
# encoded_image = model.encode(Image.open(filepath))
image_names = list(glob.glob('./*.jpg'))
print("Images:", len(image_names))
encoded_image = model.encode([Image.open(filepath) for filepath in image_names], batch_size=128, convert_to_tensor=True, show_progress_bar=True)

# Now we run the clustering algorithm. This function compares images aganist 
# all other images and returns a list with the pairs that have the highest 
# cosine similarity score
processed_images = util.paraphrase_mining_embeddings(encoded_image)
NUM_SIMILAR_IMAGES = 10 

# =================
# DUPLICATES
# =================
print('Finding duplicate images...')
# Filter list for duplicates. Results are triplets (score, image_id1, image_id2) and is scorted in decreasing order
# A duplicate image will have a score of 1.00
# It may be 0.9999 due to lossy image compression (.jpg)
duplicates = [image for image in processed_images if image[0] >= 0.999]

# Output the top X duplicate images
for score, image_id1, image_id2 in duplicates[0:NUM_SIMILAR_IMAGES]:
    print("\nScore: {:.3f}%".format(score * 100))
    print(image_names[image_id1])
    print(image_names[image_id2])

# =================
# NEAR DUPLICATES
# =================
print('Finding near duplicate images...')
# Use a threshold parameter to identify two images as similar. By setting the threshold lower, 
# you will get larger clusters which have less similar images in it. Threshold 0 - 1.00
# A threshold of 1.00 means the two images are exactly the same. Since we are finding near 
# duplicate images, we can set it at 0.99 or any number 0 < X < 1.00.
threshold = 0.99
near_duplicates = [image for image in processed_images if image[0] < threshold]

for score, image_id1, image_id2 in near_duplicates[0:NUM_SIMILAR_IMAGES]:
    print("\nScore: {:.3f}%".format(score * 100))
    print(image_names[image_id1])
    print(image_names[image_id2])